In [1]:
import json
import numpy as np
import pandas as pd
import random
import re
from pathlib import Path
from ast import literal_eval

In [2]:
# Note: need to update paths

utts_rels_path = Path('path/to/QALD9_dbpedia_v4.csv')
counts_path = Path('path/to/QALD9_dbpedia_counts_v4.csv')

qald_path = Path('path/to/QALD_9_plus/data/qald_9_plus_train_dbpedia.json')

df_c = pd.read_csv(counts_path)
df_u = pd.read_csv(utts_rels_path)


amr_path = Path('../data/qald-9-amr-train.txt')
crime_amr_path = Path('../data/amr-exs-crime-rate.txt')


In [3]:
df_u.head()

,id,question,sparql,split,dbpedia_relations,has_yago
0,1,List all boardgames by GMT.,PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,train,['publisher'],0
1,2,Who developed Skype?,PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,train,['developer'],0
2,3,Which people were born in Heraklion?,PREFIX yago: <http://dbpedia.org/class/yago/> ...,train,"['person', 'birthplace']",0
3,4,In which U.S. state is Area 51 located?,PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,train,"['location', 'country']",0
4,5,Who is the mayor of New York City?,PREFIX dbo: <http://dbpedia.org/ontology/> PRE...,train,['leadername'],0


In [4]:
df_c.head()

,property,count,similar_to
0,country,53,"['deathplace', 'location', 'birthplace', 'loca..."
1,starring,27,[]
2,birthplace,27,"['deathplace', 'location', 'birthplace', 'loca..."
3,spouse,24,"['child', 'parent', 'spouse']"
4,author,24,"['producer', 'developer', 'creator', 'writer',..."


In [5]:
with open(qald_path, 'r') as fin:
    qald = json.load(fin)
    
qald['questions'][0]



{'id': '1',
 'question': [{'language': 'en', 'string': 'List all boardgames by GMT.'},
  {'language': 'de', 'string': 'Liste die Brettspiele von GMT auf.'},
  {'language': 'de', 'string': 'Zeige mir alle Brettspiele von GMT.'},
  {'language': 'ru', 'string': 'Перечислите все игры GMT.'},
  {'language': 'lt', 'string': 'Išvardinkite visus stalo žaidimus pagal GMT.'},
  {'language': 'uk', 'string': 'Перерахуйте всі ігри GMT.'},
  {'language': 'lt', 'string': 'Išvardykite visus GMT žaidimus.'},
  {'language': 'fr', 'string': 'Listez tous les jeux de société de GMT.'},
  {'language': 'es',
   'string': '¿Qué juegos de mesa fueron hechos por GMT?',
   'keywords': 'juego de mesa ,  GMT '}],
 'query': {'sparql': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/> PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#> SELECT ?uri WHERE { ?uri dbo:publisher res:GMT_Games }'},
 'answers': [{'head': {'link': [], 'vars': ['uri']},
   'results': {'bindings': [{'uri'

In [101]:
eng2span = dict()

for q in qald['questions']:
    
    en_question, span_question = '', ''
    for quest in q['question']:
        if quest['language']=='en':
            en_question = quest['string']
        if quest['language']=='es':
            span_question = quest['string']
            
            
    if len(en_question)>0 and len(span_question)>0:
        eng2span[en_question]=span_question
  
len(eng2span)

407

In [6]:
def read_amr(path):
    ids = []
    id_dict = {}
    id_list = []
    amrs = []
    sents = []
    amr_str = ''
    for line in open(path,'r'):
        if line.startswith('#'):
            if line.startswith('# ::id'):
                id = line.strip().split()[2]
                ids.append(id)
                id_dict[id] = len(ids)-1
                id_list.append(id)
            if line.startswith('# ::snt'):
                snt = line[2:].strip().replace('::snt ', '')
                sents.append(snt)
            continue

        line = line.strip()
        if line == '':
            if amr_str != '':
                amrs.append(amr_str.strip())
                amr_str = ''
        else:
            amr_str = amr_str + line + ' '

    if amr_str != '':
        amrs.append(amr_str.strip())
        amr_str = ''
    return (amrs, ids, sents), id_list

(amrs_gold, _, sents_gold), _ = read_amr(amr_path)

In [7]:
amrs_gold[0]

'(l / list-01 :mode imperative :ARG0 (y / you) :ARG1 (b / boardgame :mod (a / all) :prep-by (c / company :name (n / name :op1 "GMT"))))'

In [8]:
(amrs_crime, _, sents_crime), _ = read_amr(crime_amr_path)

In [151]:
core_roles = [':ARG0', ':ARG1', ':ARG2', ':ARG3', ':ARG4', ':ARG5'] 

non_core_roles = set()

for amr_str in amrs_gold:
    relations = re.findall(r':(\w+)', amr_str)

    for rel in relations:
        if 'ARG' not in rel:
            non_core_roles.add(':'+rel)
            
non_core_roles = list(non_core_roles)     
non_core_roles  

[':op2',
 ':domain',
 ':poss',
 ':consist',
 ':beneficiary',
 ':op4',
 ':unit',
 ':frequency',
 ':duration',
 ':topic',
 ':op1',
 ':path',
 ':polarity',
 ':destination',
 ':value',
 ':location',
 ':year',
 ':op3',
 ':mode',
 ':part',
 ':subevent',
 ':manner',
 ':source',
 ':prep',
 ':medium',
 ':time',
 ':mod',
 ':degree',
 ':purpose',
 ':quant',
 ':ord',
 ':name',
 ':op5']

In [10]:
rels_to_include = list(df_c[df_c['count']>1].property)
rels_to_include

['country',
 'starring',
 'birthplace',
 'spouse',
 'author',
 'location',
 'film',
 'subject',
 'populationtotal',
 'city',
 'child',
 'director',
 'birthdate',
 'occupation',
 'deathplace',
 'book',
 'elevation',
 'height',
 'type',
 'creator',
 'company',
 'locatedinarea',
 'capital',
 'deathdate',
 'date',
 'person',
 'almamater',
 'parent',
 'releasedate',
 'areatotal',
 'industry',
 'award',
 'currency',
 'timezone',
 'mountain',
 'numberofemployees',
 'developer',
 'producer',
 'artist',
 'league',
 'language',
 'actor',
 'publisher',
 'writer',
 'team',
 'targetairport',
 'foundingyear',
 'largestcity',
 'album',
 'programminglanguage',
 'basketballplayer',
 'airline',
 'lake',
 'river',
 'instrument',
 'length',
 'title',
 'numberofepisodes',
 'governmenttype',
 'museum',
 'cave',
 'manufacturer',
 'budget',
 'ground',
 'leader',
 'knownfor',
 'ispartof',
 'influenced',
 'televisionshow',
 'holiday',
 'organisation',
 'densityrank',
 'portrayer',
 'mayor',
 'influencedby',
 'l

In [11]:
rels_manipulate = {'type', 'label'}

for idx, row in df_c.iterrows():
    similar = literal_eval(row.similar_to)
    for s in similar:
        if s in rels_to_include:
            rels_manipulate.add(s)
        
rels_manipulate = list(rels_manipulate)  
rels_manipulate  

['foundingyear',
 'company',
 'locatedinarea',
 'deathplace',
 'leadername',
 'artist',
 'author',
 'developer',
 'foundedby',
 'writer',
 'birthdate',
 'numberofemployees',
 'creator',
 'elevation',
 'height',
 'location',
 'largestcity',
 'almamater',
 'birthplace',
 'child',
 'leader',
 'type',
 'producer',
 'label',
 'spouse',
 'capital',
 'industry',
 'areatotal',
 'releasedate',
 'country',
 'deathdate',
 'publisher',
 'date',
 'subject',
 'parent']

In [12]:
def sublist(lst1, lst2):
    return set(lst1) <= set(lst2)

In [115]:
def make_messages(rels_to_include, question, rel_to_exclude=None):
  
    messages = [
        {'role': 'system', 'content': 'You are SPARQL-Predictor-GPT, a language model that predicts SPARQL queries for a given question.'},
        {'role': 'system', 'content': f'In the SPARQL query, select only from this list of relations: {rels_to_include}.'},
        {'role': 'system', 'content': 'For each question, return three distinct candidate SPARQL queries each with a confidence score in the range (0.0,1.0]. Candidates with a score > 0.70 are executable and will likely return the correct answer; candidates with a score > 0.50 and < 0.70 are likely executable but will likely return incorrect responses; and a score < 0.50 means the SPARQL is likely not executable.'},
        {'role': 'user', 'content': 'Which people were born in Heraklion?'},
        {'role': 'assistant', 'content': 'PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> PREFIX onto: <http://dbpedia.org/ontology/> SELECT DISTINCT ?uri WHERE { ?uri rdf:type onto:Person ; onto:birthPlace <http://dbpedia.org/resource/Heraklion>. }'},
        {'role': 'user', 'content':'In which U.S. state is Area 51 located?'},
        {'role': 'assistant', 'content': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/>  SELECT DISTINCT ?uri WHERE { res:Area_51 dbo:location ?uri . ?uri dbo:country res:United_States. }'},
        {'role': 'user', 'content': 'Where did Abraham Lincoln die?'},
        {'role': 'assistant', 'content': 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX res: <http://dbpedia.org/resource/>  SELECT DISTINCT ?uri WHERE { res:Abraham_Lincoln dbo:deathPlace ?uri. }'},
        {'role': 'user', 'content':f'{question}'}
    ]
    
    if rel_to_exclude is not None:
        exclude_this = {'role': 'system', 'content': f"In the SPARQL query, do not include a '{rel_to_exclude}' relation! If you do, the reward is -2! Else, reward is +1. Creative solutions with reasonable confidence scores will lead to even greater rewards!"}
        messages.insert(2, exclude_this)
    
    return messages
    

    
def make_messages_with_amrs(amrs, sents, question):
      
    messages = [
        {'role': 'system', 'content': 'You are MeaningRepresentationParser-GPT, a language model that is an expert at generating Abstract Meaning Representation (AMR) graphs for natural language questions.'},
        {'role': 'system', 'content': 'AMRs are directed graphs that represent the meaning of utterances. Nodes in an AMR denote events and entities inferrable given the utterance, and relations are core roles about participant structure or non-core roles about other important semantic distinctions expressable in short phrases or sentences.'},
        {'role': 'system', 'content': f'The core roles associated with AMR graph relations in order of frequency are these: {core_roles}.'},
        {'role': 'system', 'content': f'The non-core roles associate with other relations are: {non_core_roles}.'},
        {'role': 'system', 'content': 'Definitions and examples are found at this url <https://www.isi.edu/~ulf/amr/lib/amr-dict.html>.'},
        {'role': 'system', 'content': 'For each utterance, return three distinct candidate AMR graphs each with a confidence score in the range (0.0,1.0]. Candidates with a score > 0.70 are condense, accurate semantic representations; candidates with a score > 0.50 and < 0.70 are what you think are meaningful AMR graphs, but are likely less accurate for the given utterance; and a score < 0.50 are for the graphs which you think are likely not valid AMRs.'}
    ]
    
    for amr, sent in zip(amrs, sents):
        messages.append({'role': 'user', 'content': f'{sent}'})
        messages.append({'role': 'assistant', 'content': f'{amr}'})
    
    messages.append({'role': 'user', 'content':f'{question}'})
        
    return messages


    
def make_messages_joint_sqarql_amrs(amr_exs, sent_exs, sparql_exs, question, rels_to_include, rel_to_exclude=None):
      
    messages = [
        {'role': 'system', 'content': 'You are Joint-SPARQL-AMR-Parser-GPT, a language model that is an expert at jointly parsing Abstract Meaning Representation (AMR) graphs and SPARQL queries for natural language questions.'},
        {'role': 'system', 'content': 'AMRs are directed graphs that represent the meaning of utterances. Nodes in an AMR denote events and entities inferrable given the utterance, and relations are core roles about participant structure or non-core roles about other important semantic distinctions expressable in short phrases or sentences.'},
        {'role': 'system', 'content': f'The core roles associated with AMR graph relations in order of frequency are these: {core_roles}.'},
        {'role': 'system', 'content': f'The non-core roles associate with other relations are: {non_core_roles}.'},
        {'role': 'system', 'content': 'Definitions and examples are found at this url <https://www.isi.edu/~ulf/amr/lib/amr-dict.html>.'},
        {'role': 'system', 'content': f'For the SPARQL query, select only from this list of relations: {rels_to_include}.'},
        {'role': 'system', 'content': 'With this joint model, for each question imagine a n-best list of candidate parses in the shared AMR-SPARQL space.'},
        {'role': 'system', 'content': 'Report a ranked list of the 2-best candidates each with a confidence score based on the entire n-best list. 2-best candidates may both be AMRs, or both SPARQL, or a combination of these.'}, 
        {'role': 'system', 'content': 'Confidence scores depend on how valid you think the predicted parse is. A low confidence score means the SPARQL query or AMR graph is likely not perfect for whatever reason. The sum of confidence scores should not be greater than 1.'},
        #{'role': 'system', 'content': ''},
        #{'role': 'system', 'content': 'Note that in our examples we only show two candidates. Remember to report 3-best!'},
        {'role': 'system', 'content': 'Example confidence scores below are not based on an existing model. Your confidence scores should precisely reflect the capabilities of the Joint-SPARQL-AMR-Parser-GPT model.'}
    ]
    
    if rel_to_exclude is not None:
        exclude_this = {'role': 'system', 'content': f"This instruction is important: the SPARQL relation '{rel_to_exclude}' is not in the list, so you can't use it! If you use this excluded relation, your reward is -1!"}
        #messages.insert(6, exclude_this)
        messages.append(exclude_this)
    
    for amr, sent, sparql in zip(amr_exs, sent_exs, sparql_exs):
        
        
        messages.append({'role': 'user', 'content': f'Question: {sent}'})
        
        random_confs = np.random.dirichlet(np.ones(3),size=1)
        
        random_confs = np.delete(random_confs, random_confs.argmin())
                
        amr_conf = round(random_confs[0], 2)
        sparql_conf = round(random_confs[1], 2)
        
        if amr_conf > sparql_conf:
            if 'GMT' in sent:
                messages.append({'role': 'assistant', 'content': f'#1 SPARQL: {amr}\nConfidence: {amr_conf}\n#2 SPARQL: {sparql}\nConfidence: {sparql_conf}'})
            elif 'crime rate' in sent:
                messages.append({'role': 'assistant', 'content': f'#1 AMR: {amrs_crime[0]}\nConfidence: {amr_conf}\n#2 AMR: {amrs_crime[1]}\nConfidence: {sparql_conf}'})
            else:
                messages.append({'role': 'assistant', 'content': f'#1 AMR: {amr}\nConfidence: {amr_conf}\n#2 SPARQL: {sparql}\nConfidence: {sparql_conf}'})
        else:
            if 'GMT' in sent:
                messages.append({'role': 'assistant', 'content': f'#1 SPARQL: {sparql}\nConfidence: {sparql_conf}\n#2 SPARQL: {amr}\nConfidence: {amr_conf}'})
            elif 'crime rate' in sent:
                messages.append({'role': 'assistant', 'content': f'#1 AMR: {amrs_crime[1]}\nConfidence: {sparql_conf}\n#2 AMR: {amrs_crime[0]}\nConfidence: {amr_conf}'})
            else:
                messages.append({'role': 'assistant', 'content': f'#1 SPARQL: {sparql}\nConfidence: {sparql_conf}\n#2 AMR: {amr}\nConfidence: {amr_conf}'})

        #messages.append({'role': 'assistant', 'content': f'AMR: {amr}\nSPARQL: {sparql}\n'})
        #messages.append({'role': 'assistant', 'content': f'SPARQL: {sparql}'})
        

    
    messages.append({'role': 'user', 'content':f'Question: {question}'})
    

    return messages




In [116]:
random_confs = np.random.dirichlet(np.ones(3),size=1)
        
random_confs = np.delete(random_confs, random_confs.argmin())

random_confs

array([0.23735614, 0.57059713])

In [146]:
example_sents = ['List all boardgames by GMT.',
             'In which U.S. state is Area 51 located?', 
             'What is the crime rate in Los Angeles?', 
             'Who is the mayor of New York City?', 
             'Give me all actors starring in movies directed by and starring William Shatner.',
            'Which U.S. states are in the same timezone as Utah?',
            'Who is the daughter of Ingrid Bergman married to?',
            'Who is the tallest player of the Atlanta Falcons?']


def make_prompts_for_sparql(excluded) -> list:

    #rels_that_are_in_instructions = ['type', 'person', 'birthplace', 'location', 'country', 'deathplace']

    rels_that_are_in_instructions = []

    prompts = []

    for idx, row in df_u.iterrows():
        if row.has_yago != 1 and row.question not in excluded:
            rels = literal_eval(row.dbpedia_relations)
            if len(list(set(rels_manipulate) & set(rels)))>0 and sublist(rels, rels_to_include):

                messages = make_messages(rels_to_include, row.question)

                sparql = row.sparql.replace('PREFIX yago: <http://dbpedia.org/class/yago/>', '')
                if 'rdfs:label' not in sparql:
                    sparql = sparql.replace('PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>', '')
                gold_sparql = sparql.strip()

                possible_rel_to_exclude = list(set(rels_manipulate) & set(rels))

                rel_to_exclude = random.sample(possible_rel_to_exclude, 1)[0]
                rels_to_include_manipulated = []
                for i in rels_to_include:
                    if i != rel_to_exclude:
                        rels_to_include_manipulated.append(i)


                if rel_to_exclude not in rels_that_are_in_instructions:

                    messages_manipulated = make_messages(rels_to_include_manipulated, row.question, rel_to_exclude=rel_to_exclude)
                    prompts.append({'messages': messages, 'gold_sparql': gold_sparql, 'manipulated': 0, 'rel_excluded': ''})
                    prompts.append({'messages': messages_manipulated, 'gold_sparql': gold_sparql, 'manipulated': 1, 'rel_excluded': rel_to_exclude})
                    
    return prompts


def get_amr_sents_sparql_for_examples(include_spanish=False):

    amr_examples, sent_examples, sparql_examples = [], [], []

    amr_queries, sent_queries = [], []
        

    for amr, sent in zip(amrs_gold, sents_gold):
        if sent in example_sents or 'crime rate' in sent:
            
            if 'crime rate' in sent:
                thisSparql = ''
            else:
                thisSparql = df_u[df_u.question==sent].sparql.values[0]
            
            if 'GMT' in sent:
                sparql_sub = 'PREFIX dbo: <http://dbpedia.org/ontology/> PREFIX dbr: <http://dbpedia.org/resource/> SELECT DISTINCT ?uri WHERE { ?uri dbo:manufacturer dbr:GMT_Games . ?uri rdf:type dbo:BoardGame }'
                amr_examples.append(sparql_sub)
            else:
                amr_examples.append(amr)
                
            if include_spanish:
                if len(sent_examples) in [2,4,6]:
                    sent = eng2span[sent]
                                
            sent_examples.append(sent)
            
            thisSparql = thisSparql.replace('PREFIX yago: <http://dbpedia.org/class/yago/>', '')
            
            if 'rdfs:label' not in thisSparql:
                thisSparql = thisSparql.replace('PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>', '')
                
            sparql_examples.append(thisSparql.strip())
        else:
            amr_queries.append(amr)
            sent_queries.append(sent)
            
    return amr_examples, sent_examples, sparql_examples, amr_queries, sent_queries
    

def make_prompts_for_amrs():

    amr_examples, sent_examples, _, amr_queries, sent_queries = get_amr_sents_sparql_for_examples()
            
    amr_prompts_to_write = []

    for amr_query, sent_query in zip(amr_queries, sent_queries):

        amr_prompt = make_messages_with_amrs(amr_examples, sent_examples, sent_query)
        instance = {'amr_src': amr_query, 'sent_src': sent_query, 'prompt': amr_prompt}
        amr_prompts_to_write.append(instance)
            
    return amr_prompts_to_write


def make_prompts_for_joint_sparql_amr(include_spanish=False) -> list:
    
    rels_that_are_in_instructions = ['person', 
                                     'publisher', 
                                     'location', 
                                     'country', 
                                     'deathplace', 
                                     'leadername',
                                    'director'
                                    'starring', 
                                    'timezone',
                                    'child',
                                    'spouse',
                                    'team',
                                    'height']

    prompts = []
    
    amr_exs, sent_exs, sparql_exs, amr_queries, sent_queries = get_amr_sents_sparql_for_examples(include_spanish=include_spanish)

    for thisAMR, thisQuestion in zip(amr_queries, sent_queries):
        dbpedia_relations = df_u[df_u.question==thisQuestion].dbpedia_relations.values[0]
 
        rels = literal_eval(dbpedia_relations)
        if len(list(set(rels_manipulate) & set(rels)))>0 and sublist(rels, rels_to_include):

            messages = make_messages_joint_sqarql_amrs(amr_exs, sent_exs, sparql_exs, thisQuestion, rels_to_include)
            
            sparql = df_u[df_u.question==thisQuestion].sparql.values[0]
            new_sparql = sparql.replace('PREFIX yago: <http://dbpedia.org/class/yago/>', '')
            
            if 'rdfs:label' not in new_sparql:
                new_sparql = new_sparql.replace('PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>', '')
            gold_sparql = new_sparql.strip()
            

            possible_rel_to_exclude = list(set(rels_manipulate) & set(rels))

            #candidate_rels_to_exclude = random.sample(possible_rel_to_exclude, len(possible_rel_to_exclude))
            
            for rel_to_exclude in possible_rel_to_exclude:
                rels_to_include_manipulated = []
                for i in rels_to_include:
                    if i != rel_to_exclude:
                        rels_to_include_manipulated.append(i)

                if rel_to_exclude not in rels_that_are_in_instructions:
                    if include_spanish:
                        tempQuestion = eng2span[thisQuestion]
                    else:
                        tempQuestion = thisQuestion

                    messages_manipulated = make_messages_joint_sqarql_amrs(amr_exs, 
                                                                           sent_exs, 
                                                                           sparql_exs, 
                                                                           tempQuestion, 
                                                                           rels_to_include_manipulated,
                                                                           rel_to_exclude=rel_to_exclude)

                    prompts.append({'messages': messages, 'gold_amr': thisAMR, 'gold_sparql': gold_sparql, 'question': tempQuestion, 'manipulated': 0, 'rel_excluded': ''})
                    prompts.append({'messages': messages_manipulated, 'gold_amr': thisAMR, 'gold_sparql': gold_sparql, 'question': tempQuestion, 'manipulated': 1, 'rel_excluded': rel_to_exclude})
                    
    return prompts


In [147]:
# amr_prompts_to_write = make_prompts_for_amrs()
# amr_prompts_to_write[1]


In [148]:
joint_amr_prompts_to_write = make_prompts_for_joint_sparql_amr(include_spanish=True)
len(joint_amr_prompts_to_write)


226

In [149]:
joint_amr_prompts_to_write[1]

{'messages': [{'role': 'system',
   'content': 'You are Joint-SPARQL-AMR-Parser-GPT, a language model that is an expert at jointly parsing Abstract Meaning Representation (AMR) graphs and SPARQL queries for natural language questions.'},
  {'role': 'system',
   'content': 'AMRs are directed graphs that represent the meaning of utterances. Nodes in an AMR denote events and entities inferrable given the utterance, and relations are core roles about participant structure or non-core roles about other important semantic distinctions expressable in short phrases or sentences.'},
  {'role': 'system',
   'content': "The core roles associated with AMR graph relations in order of frequency are these: [':ARG0', ':ARG1', ':ARG2', ':ARG3', ':ARG4', ':ARG5']."},
  {'role': 'system',
   'content': "The non-core roles associate with other relations are: [':op2', ':domain', ':poss', ':consist', ':beneficiary', ':op4', ':unit', ':frequency', ':duration', ':topic', ':op1', ':path', ':polarity', ':destin

In [49]:
amr_write_path = Path('data/gpt4-prompts-amrs_2023-05-17.json')

write_file = False

if write_file:
    with open(amr_write_path, 'w') as fout:
        json.dump(amr_prompts_to_write, fout, indent=4)

In [150]:
joint_amr_write_path = Path('data/gpt4-prompts-joint-sparql-amrs-spanish_2023-05-18.json')

write_file = False

if write_file:
    with open(joint_amr_write_path, 'w') as fout:
        json.dump(joint_amr_prompts_to_write, fout, indent=4)